In [18]:
from mqt.bench import get_benchmark
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [19]:
n=3

In [20]:
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
qc = get_benchmark(benchmark_name="dj",
                    level="nativegates",
                    circuit_size=n,
                    compiler="qiskit",
                    compiler_settings=compiler_settings,
                    provider_name="ionq",)
print("---------------dj---------------")
print(qc.draw())
print("=================================================================================================")

---------------dj---------------
global phase: 3π/2
      ┌───────┐  ┌───────────┐ ┌────────┐  ┌──────────┐                        »
q_0: ─┤ Ry(π) ├──┤0          ├─┤ Rz(-π) ├──┤ Rx(-π/2) ├────────────────────────»
      ├───────┤  │           │ └────────┘ ┌┴──────────┤ ┌────────┐ ┌──────────┐»
q_1: ─┤ Ry(π) ├──┤  Rxx(π/2) ├────────────┤0          ├─┤ Rz(-π) ├─┤ Rx(-π/2) ├»
     ┌┴───────┴─┐│           │┌──────────┐│  Rxx(π/2) │┌┴────────┴┐└──────────┘»
q_2: ┤ Ry(-π/2) ├┤1          ├┤ Rx(-π/2) ├┤1          ├┤ Rx(-π/2) ├────────────»
     └──────────┘└───────────┘└──────────┘└───────────┘└──────────┘            »
c: 2/══════════════════════════════════════════════════════════════════════════»
                                                                               »
«      ░ ┌─┐   
«q_0: ─░─┤M├───
«      ░ └╥┘┌─┐
«q_1: ─░──╫─┤M├
«      ░  ║ └╥┘
«q_2: ─░──╫──╫─
«      ░  ║  ║ 
«c: 2/════╩══╩═
«         0  1 


In [21]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = AerSimulator()
job = sampler.run([qc], shots=1000)
result = job.result()

In [22]:
counts = result.get_counts()
print(f" > Counts: {counts}")

 > Counts: {'11': 1000}


In [26]:
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
        and expected distribution q (counts).
        Both p and q should be dictionaries: {bitstring: counts}."""
    p_sum, q_sum = sum(p.values()), sum(q.values())  # Normalize to probability distributions
    if q_sum == 0:
        raise ValueError("Expected distribution invalid (all counts = 0)")
    p_norm = {k: v / p_sum for k, v in p.items()}
    q_norm = {k: v / q_sum for k, v in q.items()}
    overlap = 0.0
    for k in set(p_norm) | set(q_norm):  # union of keys
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))  # Compute fidelity    
    return overlap**2

def rescale_fidelity(f, floor, new_floor):
    r = (1-new_floor)/(1-floor) * (f - 1) + 1
    return max(0, min(1, r))

def polarization_fidelity(counts, correct_dist, thermal_dist=None): 
    '''     counts: the measurement outcomes after `num_shots` algorithm runs
                        correct_dist: the distribution we expect to get for the algorithm running perfectly
                        thermal_dist: optional distribution to pass in distribution from a uniform superposition over all states. If `None`: generated as 
                        uniform_dist` with the same qubits as in `counts`'''
    n = len(next(iter(correct_dist)))                            # get length of random key in correct_dist to find how many qubits measured
    counts = {k.zfill(n): v for k, v in counts.items()}          # ensure that all keys in counts are zero padded to this length
    hf = hellinger_fidelity_with_expected(counts, correct_dist)   # calculate hellinger fidelity between measured expectation values and correct distribution
    if n > 16: return {'fidelity': hf, 'hf_fidelity': hf}         # to limit cpu and memory utilization, skip noise correction if more than 16 measured qubits  & if not provided, generate thermal dist based on number of qubits
    if thermal_dist is None: thermal_dist = uniform_dist(n)      # set our fidelity rescaling value as the hellinger fidelity for a depolarized state
    floor = hellinger_fidelity_with_expected(thermal_dist, correct_dist)   # rescale fidelity result so uniform superposition (random guessing) returns fidelity
      
    # rescaled to 0 to provide a better measure of success of the algorithm (polarization)return {'fidelity': rescale_fidelity(hf, floor, 0), 'hf_fidelity': hf}


In [27]:
# Build expected distribution: Example: for "constant" oracle, expected is all 0s
expected = {"000": 1000}  
# Or normalize (doesn't matter, function will normalize internally)
fidelity = hellinger_fidelity_with_expected(counts, expected)
print("Measured Counts:", counts)
print("Hellinger Fidelity:", fidelity)

Measured Counts: {'11': 1000}
Hellinger Fidelity: 0.0
